<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install --upgrade implicit

In [66]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [87]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-87-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [88]:
len(df)

2220299

In [89]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


# Data Preprocessing

In [90]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [91]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

113860

In [92]:
df[df["Qty Shipped"] == 0].head(5)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
24204,4434497,10.01.23,NaN,2023FM01,44475054,EVERIT GMBH,Business Channel,,CD71519,1830 24G 12P CLASS4 POE-STOCK,.,490,Communications & Networking,Lan Switches Managed Layer 2,Networking,"0,00",0.0
35112,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E390,TC21/TC26 HC WHITE FIVE SLOT,CHARGE ONLY CRADLE FIVE DEVICES,6026,AIDC/PoS Accessories & Supplies,Docking Station,"Other (incl. AIDC/POS, V7)","0,00",0.0
35113,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E412,TC21/26 BELT CLIP HC,DISINFECTANT READY TC21/TC26 HC,5706,AIDC/PoS Barcode Scanners,Barcode Scanner Accessories,"Other (incl. AIDC/POS, V7)","0,00",0.0
40384,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58182,IPHONE 14 PRO 128GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40385,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58183,IPHONE 14 PRO 128GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0


In [93]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [94]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [95]:
len(df)

685862

In [96]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

## User and sku unqiue list

In [97]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [98]:
df_1 = df[["bcn_id","sku_id","Qty Shipped", "purchase"]]

In [99]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685862 entries, 7402 to 2220294
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   bcn_id       685862 non-null  int16  
 1   sku_id       685862 non-null  int32  
 2   Qty Shipped  685862 non-null  float64
 3   purchase     685862 non-null  int64  
dtypes: float64(1), int16(1), int32(1), int64(1)
memory usage: 19.6 MB


In [100]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "purchase": "sum"
}).reset_index()

In [101]:
# add column that represent sku count
grouped_df['sku_count'] = grouped_df.groupby('bcn_id')['sku_id'].transform('nunique')

In [102]:
# drop customers with less than 2 product as loocv will not work
len(grouped_df)
grouped_df = grouped_df[grouped_df.sku_count >= 2]
grouped_df = grouped_df[["bcn_id", "sku_id", "purchase"]]
grouped_df

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,2
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284186,9880,26029,1
284187,9880,30295,1
284188,9880,42782,3
284193,9885,1816,2


In [103]:
grouped_df_binary = grouped_df[["bcn_id", "sku_id", "purchase"]]
grouped_df_binary["purchase"] = 1
grouped_df_binary

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,1
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284186,9880,26029,1
284187,9880,30295,1
284188,9880,42782,1
284193,9885,1816,1


## LOOCV

with K=1 so 1 item per user gets dropped out

In [104]:
def train_test_split(df, holdout_num):
    """ perform training testing split

    @param df: dataframe
    @param holdhout_num: number of items to be held out per user as testing items

    @return df_train: training data
    @return df_test testing data

    """
    # perform deep copy to avoid modification on the original dataframe
    df_train = df.copy(deep=True)
    df_test = df.copy(deep=True)

    # get test set
    df_test = df_test.groupby(['bcn_id']).head(holdout_num).reset_index()

    # get train set
    df_train = df_train.merge(
        df_test[['bcn_id', 'sku_id']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', 1).reset_index(drop=True)

    # drop index
    df_test = df_test.drop(columns=["index"])

    # sanity check to make sure we're not duplicating/losing data
    assert len(df) == len(df_train) + len(df_test)

    return df_train, df_test

In [105]:
df_train_binary, df_test_binary = train_test_split(grouped_df_binary, 1)

<ipython-input-104-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [106]:
df_train, df_test = train_test_split(grouped_df, 1)

<ipython-input-104-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [107]:
df_train_binary

,bcn_id,sku_id,purchase
0,0,2277,1
1,0,2796,1
2,0,4417,1
3,0,5535,1
4,0,6557,1
...,...,...,...
274304,9880,8551,1
274305,9880,26029,1
274306,9880,30295,1
274307,9880,42782,1


### Stratifying for same bcns in train test

In [108]:
# check whether there are users in test but not in train
print(len(df_train.bcn_id.unique()), len(df_test.bcn_id.unique()))
print(len(df_train_binary.bcn_id.unique()), len(df_test_binary.bcn_id.unique()))

7378 7378
7378 7378


In [109]:
# check whether there are skus in test but not in train
print(len(df_train.sku_id.unique()), len(df_test.sku_id.unique()))
print(len(df_train_binary.sku_id.unique()), len(df_test_binary.sku_id.unique()))

48172 4088
48172 4088


In [110]:
train_sku_ids = set(df_train.sku_id.unique())
test_sku_ids = set(df_test.sku_id.unique())
test_sku_ids.issubset(train_sku_ids)

False

In [111]:
test_skus_ids = pd.DataFrame(test_sku_ids,columns=["sku_id"])
test_skus_ids["in_train"] = test_skus_ids.isin(train_sku_ids)
test_skus_ids

,sku_id,in_train
0,0,False
1,1,True
2,2,True
3,3,True
4,4,True
...,...,...
4083,24557,True
4084,32751,True
4085,32760,True
4086,24569,True


In [112]:
df_train[df_train.sku_id == 32766]

,bcn_id,sku_id,purchase


In [62]:
def stratify_bcn(train_df, test_df):
  """ remove users from test set, that are not in train
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  trained_bcn_ids = train_df['bcn_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  stratified_df_test = test_df[test_df['bcn_id'].isin(trained_bcn_ids)]

  return stratified_df_test

In [113]:
def stratify_sku(train_df, test_df):
  """ remove skus from test set that are not in train set
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  train_bcn_ids = train_df['sku_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  sku_stratified_df_test = test_df[test_df['sku_id'].isin(train_bcn_ids)]

  return sku_stratified_df_test

In [114]:
stratified_df_test = stratify_bcn(df_train, df_test)

In [115]:
sku_stratified_df_test = stratify_sku(df_train, stratified_df_test)

In [116]:
len(sku_stratified_df_test)

6611

In [118]:
train_sku_ids_2 = set(df_train.sku_id.unique())
test_sku_ids = set(sku_stratified_df_test.sku_id.unique())
test_sku_ids.issubset(train_sku_ids_2)

True

In [119]:
stratified_df_test_binary = stratify_bcn(df_train_binary, df_test_binary)

In [120]:
sku_stratified_df_test_binary = stratify_sku(df_train_binary, stratified_df_test_binary)

In [121]:
len(sku_stratified_df_test_binary)

6611

### Negative sampling


In [122]:
def negative_sampling(bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    return df_neg

In [123]:
neg_train = negative_sampling(
    bcn_ids=df_train.bcn_id.unique(),
    sku_ids=df_train.sku_id.unique(),
    items=grouped_df_binary.sku_id.unique(),
    n_neg=5
)

In [124]:
# create final training and testing sets
#df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
df_train = pd.concat([df_train, neg_train], ignore_index=True)

In [125]:
df_train_binary = pd.concat([df_train_binary, neg_train], ignore_index=True)

## Create sparse user x item and item x user matrices

In [126]:
def create_user_item_csr_matrix(df_pl):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_user_item = sparse.csr_matrix((df_pl['purchase'].astype(float), (df_pl['bcn_id'], df_pl['sku_id'])))

  return sparse_user_item

In [127]:
def create_item_user_csr_matrix(df_pl):
  """ create a sparse data matrix with scipy sparse matrix
      @return sparse user x item matrix
  """
  sparse_item_user = sparse.csr_matrix((df_pl['purchase'].astype(float), (df_pl['sku_id'], df_pl['bcn_id'])))

  return sparse_item_user

In [130]:
sparse_user_item_train = create_user_item_csr_matrix(df_train)
sparse_user_item_test = create_user_item_csr_matrix(sku_stratified_df_test)

In [131]:
sparse_user_item_train_binary = create_user_item_csr_matrix(df_train_binary)
sparse_user_item_test_binary = create_user_item_csr_matrix(sku_stratified_df_test_binary)

In [132]:
sparse_item_user_train = create_item_user_csr_matrix(df_train)
sparse_item_user_test = create_item_user_csr_matrix(sku_stratified_df_test)

In [133]:
sparse_item_user_train_binary = create_item_user_csr_matrix(df_train_binary)
sparse_item_user_test_binary = create_item_user_csr_matrix(sku_stratified_df_test_binary)

In [156]:
user_items = create_user_item_csr_matrix(grouped_df)
user_items

<9886x49112 sparse matrix of type '<class 'numpy.float64'>'
	with 281687 stored elements in Compressed Sparse Row format>

##Create unique sku and user lists

In [134]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()

In [135]:
users = df[["bcn_id", "BranchCustomerNbr"]].drop_duplicates()

# **ALS Model**

In [136]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## ALS Model param tuning

In [137]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(sparse_user_item_train)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 2.82 s, sys: 4.51 s, total: 7.33 s
Wall time: 4.42 s


ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





In [ ]:
# def validate(sparse_user_item_train, sparse_user_item_test, factors=200, alpha=40, iterations=20, regularization=0.01, show_progress=True):
#     """ Train an ALS model with <<factors>> (embeddings dimension)
#     for <<iterations>> over matrices and validate with MAP@10
#     """

#     model = implicit.als.AlternatingLeastSquares(factors=factors,
#                                                  alpha=alpha,
#                                                  iterations=iterations,
#                                                  regularization=regularization)

#     model.fit(sparse_user_item_train, show_progress=show_progress)

#     # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
#     # TODO: change MAP@10 to a library that allows repeated items in prediction
#     map10 = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, show_progress=show_progress, num_threads=4)
#     print(f"Factors: {factors:>3} - Confidence: {alpha:>9} - Regularization: {regularization:4.3f} - Iterations: {iterations:>2} ==> MAP@10: {map10:6.5f}")
#     return map10

In [ ]:
# %%time
# best_map10 = 0
# for factors in [10, 50, 100, 150, 200]:
#   for alpha in [10, 20, 30, 40]:
#     for iterations in [10, 15, 20]:
#       for regularization in [0.01, 0.05, 0.1]:
#           map10 = validate(sparse_user_item_train, sparse_user_item_test, factors, alpha, iterations, regularization, show_progress=False)
#           if map10 > best_map10:
#             best_map10 = map10
#             best_params = {'factors': factors, 'confidence': alpha, 'iterations': iterations, 'regularization': regularization}
#             print(f"Best MAP@10 found. Updating: {best_params}")

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV


class WrappedALS(BaseEstimator):
    def __init__(self, factors=100, regularization=0.01, iterations=20, alpha=40):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.alpha = alpha
        self.als_model = AlternatingLeastSquares(
            factors=self.factors,
            regularization=self.regularization,
            iterations=self.iterations,
            calculate_training_loss=True,
        )

    def fit(self, X, y=None):
        self.als_model.fit(X)
        return self

    def predict(self, X):
        return self.als_model.user_factors.dot(self.als_model.item_factors.T)

    def score(self, X, y=None):
        precision = precision_at_k(self.als_model, X, X.T, K=10)
        return precision_at_k

# Define the hyperparameter ranges
param_grid = {
    'factors': [10, 50, 100, 150, 200],
    'alpha': [40],
    'iterations': [10, 20],
    'regularization': [0.01, 0.05]
}

# Initialize WrappedALS model
wrapped_als_model = WrappedALS()

# Initialize GridSearchCV
grid_search = GridSearchCV(wrapped_als_model, param_grid, cv=3, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(sparse_user_item_train)

# Get the best ALS model and its hyperparameters
best_als_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:")
print(best_params)

print("-----------------------------------------------------------------------------------------")
print("Best model MAP@10:")
best_model = AlternatingLeastSquares(**best_params)
map_10_best = mean_average_precision_at_k(best_model, sparse_user_item_train, sparse_user_item_test, K=10)
print(map_10_best)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: ignored

In [140]:
sparse_user_item_train

<9886x49112 sparse matrix of type '<class 'numpy.float64'>'
	with 311168 stored elements in Compressed Sparse Row format>

In [141]:
sku_stratified_df_test

,bcn_id,sku_id,purchase
0,0,2276,1
1,1,1424,1
2,2,1908,1
3,3,17537,8
4,5,2798,1
...,...,...,...
7373,9866,34264,1
7374,9875,6679,1
7375,9877,42140,1
7376,9880,2876,2


In [142]:
%%time
model_1 = implicit.als.AlternatingLeastSquares(factors=100,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1, sparse_user_item_train, sparse_user_item_test, K=10)
print(map10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6611 [00:00<?, ?it/s]

0.05388710014333962
CPU times: user 57.2 s, sys: 1min 2s, total: 1min 59s
Wall time: 1min 14s


In [143]:
model_2 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_2.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_2,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6611 [00:00<?, ?it/s]

0.0609780956702753


In [145]:
model_3 = AlternatingLeastSquares(factors=150, alpha=40, regularization=0.05,
                                  iterations=20, calculate_training_loss=True)
model_3.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_3,sparse_user_item_train, sparse_user_item_test, K=10)
map10

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6611 [00:00<?, ?it/s]

0.06043246825276767

In [146]:
model_4 = AlternatingLeastSquares(factors=150, alpha=40, regularization=0.05,
                                  iterations=20, calculate_training_loss=True)
model_4.fit(sparse_user_item_train_binary)
map10 = mean_average_precision_at_k(model_4,sparse_user_item_train_binary, sparse_user_item_test_binary, K=10)
map10

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6611 [00:00<?, ?it/s]

0.05825169930827159

## ALS Best Model Performance

In [147]:
precision_als = precision_at_k(model_3, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

  0%|          | 0/6611 [00:00<?, ?it/s]

0.12554832854333686

In [148]:
ndcg = ndcg_at_k(model_3, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

  0%|          | 0/6611 [00:00<?, ?it/s]

0.07566750354043612

## ALS Model Recommendations

In [176]:
 # Assuming you want recommendations for user with ID 10
user_id = 275

# Now you can call the recommend function
recommended = model_3.recommend(user_id, user_items[user_id], N=13, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

,sku_id,conf,Sku,Product Descr1,Product Descr2,ProductGroupDescription
0,22954.0,0.766616,CC47067,TRELLIX PROTECT STANDARD,COM SUB 1Y SUB BS A:5-250,Other Security Software
1,13633.0,0.620980,677QL89,CATALYST 9200L 24-PORT POE+,4 X 10G NETWORK ESSENTIALS,Lan Switches Managed Layer 2
2,22965.0,0.567549,CC47441,SEC FOR MS EXCH EPO 1:1BZ,COM SUB 1Y SUB BS A:5-250,Other Security Software
3,13652.0,0.523446,677QP85,600W AC CONFIG 5,POWER SUPPLY,Power Supplies
4,13452.0,0.445097,677NW74,AIRONET DNA ESSENTIALS,3 YEAR TERM LICENSE,Network Management & Utility Software
5,13953.0,0.427214,677SN67,CATALYST 802.11AX AP EXT. ANTEN,4X4:4 MIMO IOT BT5MGIG RHL-E DOM,Wireless Lan Access Points (Ap)
6,13804.0,0.417205,677RP77,CATALYST 802.11AX APINT ANTENNA,4X4:4 MIMO IOT BT5MGIG USB E DOM,Wireless Lan Access Points (Ap)
7,23154.0,0.409915,CC71596,SEC FOR MS EXCH EPO 1:1BZ,GOV SUB 1Y SUB BS B:251-1000,Other Security Software
8,13769.0,0.407640,677RG09,CISCO CATALYST 802.11AX AP INT,ANTENNA 4X4:4 MIMO BT 5 MGIG,Wireless Lan Access Points (Ap)
9,13471.0,0.404087,677P577,1000BASE-LX/LH SFP TRANSCEIVER,MODULE MMF/SMF 1310NM DOM,"Transceivers (Gbic, Spf) & Media Converters"


In [175]:
sku_stratified_df_test[200:400]

,bcn_id,sku_id,purchase
232,269,1450,1
234,271,17995,1
235,273,10156,1
236,274,20522,1
237,275,13643,1
...,...,...,...
471,606,7753,1
472,608,9709,1
473,609,24502,1
474,610,690,1


In [177]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

,bcn_id,sku_id,purchase
237,275,13643,1


In [178]:
user_purchases = pd.DataFrame(df[df.bcn_id == user_id][["Product Descr1","Sku", "ProductGroupDescription", "purchase"]].groupby(
                              ["Sku","Product Descr1", "ProductGroupDescription"])["purchase"].sum().sort_values(ascending=False))
user_purchases[0:20]

,,,purchase
Sku,Product Descr1,ProductGroupDescription,
CB32850,DNA ESSENTIALS 3 YEAR LICENSE,Network Management & Utility Software,3
677SB70,C9300L CISCO DNA ESSENTIALS,Network Management & Utility Software,2
CC70881,TRELLIX STANDARD 1:1BZ,Other Security Software,2
CC71595,SEC FOR MS EXCH EPO 1:1BZ,Other Security Software,2
CE92527,SOLN SUPP SW SUB C9300 DNA,Services And Warranties,1
677QP75,C9200L DNA ESSENTIALS 24-PORT,Network Management & Utility Software,1
CC72388,COMP DATA PROT ADV P:1BZP+,Other Security Software,1
677QZ00,715W AC 80+ PLATINUM CONFIG 1,Power Supplies,1
CB32957,CATALYST 9300X 24X25G FIBER,Lan Switches Managed Layer 3,1


In [179]:
#Get similar items
item_id = 40
n_similar = 10
similar = model_2.similar_items(item_id, n_similar)
print(similar)

(array([36081, 33909, 28762, 27736, 26413,  7125,   790, 27609, 26473,
        7808], dtype=int32), array([1.        , 1.        , 1.        , 1.        , 0.99999994,
       0.99999994, 0.99999994, 0.9999999 , 0.9999999 , 0.9999999 ],
      dtype=float32))


# BPR Model

In [181]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808298 sha256=81cfe330993df21442639f92ff6ef44e554a663825b1c7dceee9ed5a502d45cb
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [182]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.model_selection import train_test_split
from lightfm.datasets import fetch_movielens

## Implicit BPR

In [183]:
model_bpr = BPR(factors=200, learning_rate=0.01, regularization=0.01,
                verify_negative_samples=True)

In [184]:
model_bpr.fit(sparse_user_item_train_binary)

  0%|          | 0/100 [00:00<?, ?it/s]

In [191]:
auc = AUC_at_k(model_bpr, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10, show_progress=True)
auc

  0%|          | 0/6611 [00:00<?, ?it/s]

0.5070077130105727

In [193]:
map10 = mean_average_precision_at_k(model_bpr, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10, show_progress=True,
                                    num_threads=10)
map10

  0%|          | 0/6611 [00:00<?, ?it/s]

0.0043608175887709984

In [200]:
 # Assuming you want recommendations for user with ID 10
user_id = 275

# Now you can call the recommend function
recommended = model_bpr.recommend(user_id, user_items[user_id], N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab.sort_values(by="conf", ascending=False)

,sku_id,conf,Sku,Product Descr1,Product Descr2,ProductGroupDescription
0,10965.0,0.856612,54153C6,WORRY FREE 5 SERVICES ADV,RNW LIZ 12 M - 0011 - 0025 USER,Other Security Software
1,43129.0,0.827776,N170825,LANCOM GS-3126XP,MANAGED LAYER-3-SWITCH 24 PORTS,Lan Switches Managed Layer 3
2,43101.0,0.818499,N170742,LANCOM ADVANCED VPN CLIENT,WIN,Network Management & Utility Software
3,37568.0,0.798571,CG25158,BU+REPL ENT COM,PR MIG FR BRE SB,Back-Up Software
4,11065.0,0.794843,5417HQ3,WORRY FREE 9 STANDARD,ADD LIZ 12 M - 0026 - 0050 USER,"Virus Detection, Antivirus Software"
5,24357.0,0.791499,CD12272,POWEREDGE R350 SERVER,CTO CONFIG,Servers
6,29334.0,0.791086,CE57605,FRACHTKOSTEN / FREIGHT,,Vendor Specific Configuation Fees
7,14868.0,0.791080,803TS37,OFFICE STD,OLV SA NL 1YACQY1 ADP,Services And Warranties
8,43986.0,0.790560,T130SC0,CYBER BACKUP STD SERVER,1Y RNW PCS ESD,Disaster/File Recovery Software
9,28554.0,0.770177,CE40398,AUTOCAD LT 2023 NEW,SGL-US 3YR SUBSCRIPTION,Cad/Cam/Cae Software


In [201]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

,bcn_id,sku_id,purchase
237,275,13643,1


## LightFM BPR

In [ ]:
dataset = Dataset()
dataset.fit(user_ids, sku_ids)

In [ ]:
movielens = fetch_movielens()
movielens

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 90570 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 9430 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [ ]:
train = movielens["train"]
pd.DataFrame(train.toarray())

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
939,0,0,0,2,0,0,4,5,3,0,...,0,0,0,0,0,0,0,0,0,0
940,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
item_feature_lables = movielens["item_feature_labels"]
item_feature_lables

array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')

In [ ]:
interactions, weights = dataset.build_interactions(zip(user_ids, sku_ids))

In [ ]:
# Split the interactions matrix into train and test sets
train_interactions, test_interactions = train_test_split(interactions, test_size=0.2, random_state=42)

In [ ]:
model = LightFM(learning_rate=0.05, loss='bpr')

In [ ]:
model.fit(train_interactions, epochs=10)

In [ ]:
sku_stratified_df_test[sku_stratified_df_test.bcn_id == user_id]

## BPR Model param tuning